In [1]:
import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout
import mindspore
import argparse
import numpy as np
import logging
import mindspore.dataset as ds
import os
import re

import json

from tqdm import tqdm
from datetime import datetime
from mindspore.nn import CrossEntropyLoss
from mindspore import nn, ops
from mindspore.train.serialization import save_checkpoint
from mindspore.dataset import TextFileDataset, GeneratorDataset

from mindnlp.transforms import BertTokenizer
from mindnlp.modules import Accumulator
from mindnlp.models import GPT2Config, GPT2LMHeadModel

/root/miniconda3/lib/python3.8/site-packages/mindnlp/utils/download.py:29: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
epochs = 20
batch_size = 8

lr = 1e-4
warmup_steps = 2000
accumulate_step = 2
max_grad_norm = 1.0

log_step = 100

In [9]:
from mindnlp._legacy.amp import auto_mixed_precision
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

config = GPT2Config(vocab_size=len(tokenizer))
model = GPT2LMHeadModel(config, ignore_index=tokenizer.pad_token_id)
# model = auto_mixed_precision(model, 'O1')

optimizer = nn.AdamWeightDecay(model.trainable_params(), lr)
accumulator = Accumulator(optimizer, accumulate_step, max_grad_norm)
params = mindspore.load_checkpoint('./gpt_epoch_finetune_5.ckpt')
mindspore.load_param_into_net(model, params) 

[]

In [4]:
def clean_text(text):
    # cleaned_text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9,.]', ' ', text)
    pattern = r"[^a-zA-Z0-9\u4e00-\u9fa5,.\?!，。？、]"
    cleaned_text = re.sub(pattern, " ", text)
    return cleaned_text.replace('"', '').replace('\t', '')

In [5]:
inputBox = widgets.Textarea(description='Question:',layout=Layout(width='60%'))
queryBtn = widgets.Button(description='query',button_style='primary',icon='search',layout=Layout(width='20%', height='40px'))
outputBox = widgets.Textarea(
    value='',
    description='Answer:',
    disabled=False,
    layout=Layout(width='80%')
)

In [6]:
def robot_query(query):
    summury = ''

    max_summary_len = 300
    query = clean_text(query)
    article = f'{query}<Paragraph>'
    show = article.replace('<Paragraph>', '<sep>')
    input_ids = tokenizer.encode(article).ids
    print(f'query: \n{show}\n\nanswer:')

    for _ in range(max_summary_len):
        inputs = mindspore.Tensor(input_ids, mindspore.int32)
        output = model(inputs)[0]
        pred = output.argmax(-1)[-1]
        input_ids.append(pred.asnumpy())
        summury = summury + tokenizer.id_to_token(pred.asnumpy())

        if summury[-1] == ']':
            if summury[-2]!='。':
#                 print('。', end='')
                outputBox.value = outputBox.value + '。'
            break

#         print(summury[-1], end='')
        outputBox.value = outputBox.value + summury[-1]

In [7]:
display(widgets.HBox([inputBox,queryBtn]),outputBox)

def on_button_clicked(b):
#     inputBox.value = inputBox.value + 'hhh'
    outputBox.value = ''
    robot_query(inputBox.value)

queryBtn.on_click(on_button_clicked)

Textarea(value='', description='Answer:', layout=Layout(width='80%'))